In [ ]:
# if run on colab
#!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

In [ ]:
# if run on kaggle
!pip install torcheeg

In [ ]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [ ]:
# if run on kaggle
root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

# if run on colab
# root_data_path = './BCICIV-2a-mat/'

In [ ]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    # skip_trial_with_artifacts=True,
    # offline_transform=transforms.Compose([
    #     transforms.BandDifferentialEntropy(apply_to_baseline=True),
    #     transforms.To2d(apply_to_baseline=True),
    #     transforms.ToTensor(apply_to_baseline=True)
    # ]),
    online_transform=transforms.Compose([
        # transforms.To2d(apply_to_baseline=True),
        # transforms.ToTensor(apply_to_baseline=True),
        transforms.To2d(),
        transforms.ToTensor(),
        # transforms.CWTSpectrum(apply_to_baseline=True),
        # transforms.BandDifferentialEntropy(apply_to_baseline=True),
        # transforms.BaselineRemoval(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

In [ ]:
print("Dataset's info: ")
print(dataset.info)

In [ ]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [ ]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )
    
    model = ATCNet(
        num_classes=4,
        num_electrodes=22,
        chunk_size=7*250,
    )
    
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=500,
        default_root_dir=f'./examples_pipeline/atcnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

In [ ]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))